# **Projeto 2 - Introdução à Inteligência Artificial (2025/1)**

- **Aluno(a):** Matheus Duarte da Silva - 211062277
- **Professor:** Díbio Leandro Borges

## **Objetivo do Projeto**
Este projeto visa implementar e avaliar um método de Deep Learning para a contagem de árvores em uma cidade, com base no artigo de referência de [Zamboni et al. (2021)](https://github.com/pedrozamboni/individual_urban_tree_crown_detection).
Os resultados serão comparados com os publicados no artigo, utilizando as mesmas métricas.


## Configuração Inicial

Conectando ao ambiente do Google Drive, aonde está localizado todas as pastas importantes para o projeto, como dataset, repositório, pasta de resultados e afins.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

### Definindo diretório de trabalho

Aqui estamos definindo o local do Drive em que todo o projeto será localizado, essa parte é importante para manter a organização de todos os arquivos.

In [ ]:
import os

# Caminhos importantes
%cd /content/drive/MyDrive/iia-trabalho-2/
project_path = "/content/drive/MyDrive/iia-trabalho-2/"

# Mudar para o diretório do YOLO-MS
try:
    os.chdir(project_path)
    print(f"Diretório de trabalho alterado para: {os.getcwd()}")

except FileNotFoundError:
    print(f"ERRO: O diretório '{project_path}' não foi encontrado.")
    print("Verifique se o caminho está correto e se a pasta do projeto foi compartilhada com você.")

### Instalando bibliotecas

Esta sendo definido as versões compatíveis das bibliotecas `PyTorch`, `NumPy` e suas dependências.

Para a biblioteca principal para o projeto será utilizado a `ultralytics` que contém algumas implementações do modelo YOLO, que será o modelo para detecção de objetos escolhido.

In [ ]:
# Instalando as dependências principais para o modelo
!pip install numpy==2.0.2 torch==2.6.0 torchvision==0.21.0 torchaudio==2.6.0

# Célula 4: Instalar a biblioteca
!pip install -q ultralytics

### Convertendo Dataset

A maneira em que o dataset fornecido pelo artigo de detecção de árvores em imagens RGB não reconhecido pelo YOLOv11, modelo que será utilizado para o projeto, assim, é preciso organizar os dados de uma maneira que seja reconhecido.

A célula abaixo define métodos para realizar a conversão do dataset original para um dataset que seja reconhecido para o treinamento.

In [ ]:
import os
import shutil
from PIL import Image

def convert_bbox_to_yolo(img_width, img_height, bbox):
    """
    Converte um bounding box do formato [x_min, y_min, x_max, y_max]
    para o formato YOLO [x_center, y_center, width, height] normalizado.
    """
    x_min, y_min, x_max, y_max = bbox

    # Calcula as dimensões e o centro do bounding box em píxeis
    box_width = x_max - x_min
    box_height = y_max - y_min
    x_center = x_min + (box_width / 2)
    y_center = y_min + (box_height / 2)

    # Normaliza as coordenadas (divide pelo tamanho da imagem)
    x_center_norm = x_center / img_width
    y_center_norm = y_center / img_height
    width_norm = box_width / img_width
    height_norm = box_height / img_height

    return x_center_norm, y_center_norm, width_norm, height_norm


def process_dataset_split(split_name, fold_path, img_source_path, bbox_source_path):
    """
    Processa um split (train, val, or test), copiando imagens e convertendo labels.
    """
    print(f"A processar o conjunto: {split_name}...")

    # Caminhos para as listas de imagens de treino/validação/teste
    img_list_file = os.path.join(fold_path, f'{split_name}.txt')

    if not os.path.exists(img_list_file):
        print(f"Aviso: Ficheiro de lista '{img_list_file}' não encontrado. A pular este conjunto.")
        return

    # Cria as pastas de destino para este split
    dest_img_path = os.path.join(YOLO_DATASET_PATH, 'images', split_name)
    dest_label_path = os.path.join(YOLO_DATASET_PATH, 'labels', split_name)
    os.makedirs(dest_img_path, exist_ok=True)
    os.makedirs(dest_label_path, exist_ok=True)

    with open(img_list_file, 'r') as f:
        image_names = [line.strip() for line in f.readlines()]

    for img_name in image_names:
        # Define os caminhos de origem e destino para a imagem
        img_filename_base = os.path.splitext(img_name)[0]
        source_img = os.path.join(img_source_path, img_name)
        dest_img = os.path.join(dest_img_path, img_name)

        # Copia a imagem
        if os.path.exists(source_img):
            shutil.copy(source_img, dest_img)
        else:
            print(f"Aviso: Imagem '{source_img}' não encontrada. A pular.")
            continue

        # Processa as labels
        bbox_file = os.path.join(bbox_source_path, f'{img_filename_base}.txt')
        yolo_label_file = os.path.join(dest_label_path, f'{img_filename_base}.txt')

        if os.path.exists(bbox_file):
            # Obtém as dimensões da imagem
            with Image.open(source_img) as img:
                img_width, img_height = img.size

            with open(bbox_file, 'r') as bf, open(yolo_label_file, 'w') as yf:
                for line in bf:
                    parts = list(map(int, line.strip().split()))
                    if len(parts) == 4:
                        # Converte e escreve a linha no formato YOLO
                        yolo_coords = convert_bbox_to_yolo(img_width, img_height, parts)
                        # Assume classe 0 para todos os objetos
                        yf.write(f"0 {yolo_coords[0]:.6f} {yolo_coords[1]:.6f} {yolo_coords[2]:.6f} {yolo_coords[3]:.6f}\n")
    print(f"Conjunto {split_name} processado com sucesso!")


def main():
    """Função principal para orquestrar a conversão."""
    print("Iniciando a conversão do dataset para o formato YOLO...")

    # Caminhos de origem
    img_list_path = os.path.join(ORIGINAL_DATASET_PATH, 'img_list', str(FOLD_NUMBER))
    rgb_path = os.path.join(ORIGINAL_DATASET_PATH, 'rgb')
    bbox_path = os.path.join(ORIGINAL_DATASET_PATH, 'bbox_txt')

    # Remove a pasta de destino se já existir para começar do zero
    if os.path.exists(YOLO_DATASET_PATH):
        shutil.rmtree(YOLO_DATASET_PATH)
        print(f"Pasta existente '{YOLO_DATASET_PATH}' removida.")

    # Processa cada conjunto (train, val, test)
    process_dataset_split('train', img_list_path, rgb_path, bbox_path)
    process_dataset_split('val', img_list_path, rgb_path, bbox_path)
    process_dataset_split('test', img_list_path, rgb_path, bbox_path)

    # Cria o ficheiro dataset.yaml
    yaml_content = f"""
# Caminho para as pastas de treino e validação (relativo ao ficheiro .yaml)
train: ./images/train
val: ./images/val
test: ./images/test

# Número de classes
nc: 1

# Nomes das classes
names: ['{CLASS_NAME}']
"""
    with open(os.path.join(YOLO_DATASET_PATH, 'dataset.yaml'), 'w') as f:
        f.write(yaml_content)

    print("\nConversão concluída com sucesso!")
    print(f"O seu dataset em formato YOLO está pronto na pasta: '{YOLO_DATASET_PATH}'")

#### Configurando o Dataset

In [ ]:
CLASS_NAME = 'arvore'

# Caminho para a pasta principal do dataset
ORIGINAL_DATASET_PATH = '/content/drive/MyDrive/iia-trabalho-2/datasets'

# Número da sub-pasta dentro do `img_list`
FOLD_NUMBER = 4

# Caminho da pasta onde o novo dataset em formato YOLO será criado
YOLO_DATASET_PATH = f'/content/drive/MyDrive/iia-trabalho-2/datasets/yolo_dataset_{FOLD_NUMBER}'

main()

## Configurações para o Treino do Modelo

Aqui esta sendo definido todos os parâmetros necessários para os treinamentos, seguindo a metodologia e os requisitos necessários do artigo base.

In [ ]:
import ultralytics

# Modelo base do YOLOv11 a ser usado
# Opções: 'yolo11n.pt' (nano), 'yolo11s.pt' (small), 'yolo11m.pt' (medium)
MODEL_NAME = 'yolo11n.pt'

# Caminho para a pasta do novo dataset com suas configurações
DATA_YAML_PATH = f'/content/drive/MyDrive/iia-trabalho-2/{YOLO_DATASET_PATH}/dataset.yaml'

# Número de épocas
EPOCHS = 24

# Tamanho das imagens de entrada para o modelo
IMAGE_SIZE = 512

# Nome da pasta principal onde os resultados serão guardados
PROJECT_NAME = f'/content/drive/MyDrive/iia-trabalho-2/results/treinamento_{FOLD_NUMBER}'

# Nome para cada run do treino, muda de acordo com a sub-pasta do `img_list`
RUN_NAME = f'arvore_detector_run1_fold{FOLD_NUMBER}_epch_24'

print("✅ Configurações de treino carregadas.")
print(f"   - Modelo: {MODEL_NAME}")
print(f"   - Épocas: {EPOCHS}")
print(f"   - Nome da Experiência: {RUN_NAME}")

## Treinando o Modelo

Com os parâmetros definidos, iniciamos o treinamento do modelo.

In [ ]:
from ultralytics import YOLO

print("Iniciando o carregamento do modelo...")
model = YOLO(MODEL_NAME)

print(f"🚀 Iniciando o treino para a sub-pasta {FOLD_NUMBER}...")
results = model.train(data=DATA_YAML_PATH, epochs=EPOCHS,
                      imgsz=IMAGE_SIZE, project=PROJECT_NAME, name=RUN_NAME)

print(f"Treino concluído! Resultados guardados em: {PROJECT_NAME}/{RUN_NAME}")

## Validação do Modelo

Carregamos os pesos do melhor modelo treinado (`best.pt`) e o avaliamos no conjunto de teste, que não foi visto durante o treino.

In [ ]:
# Caminho para os pesos obtidos no treinamento anterior
MODEL_WEIGHTS_PATH = f'{PROJECT_NAME}/{RUN_NAME}/weights/best.pt'

print(f"Carregando modelo: {RUN_NAME}")
model = YOLO(MODEL_WEIGHTS_PATH)

print("Iniciando a validação no conjunto de teste...")
metrics = model.val(data=DATA_YAML_PATH)

print("✅ Validação concluída!")